In [ ]:
# PASTE THIS TO THE FIRST CELL OF THE NOTEBOOK IN ORDER TO HAVE WORKING IMPORTS
import sys
import os
current_dir = os.getcwd()
parent_parent_dir = os.path.abspath(os.path.join(current_dir, '../..')) # tweak so that you get dir of code project

sys.path.append(parent_parent_dir)


In [ ]:
import numpy as np
import pandas as pd
from src.features.get_x_y_tuples import *
import ast
import matplotlib.pyplot as plt

In [ ]:
# 1. load into dataframe
df_k729_2022_cuid = pd.read_csv('../../data/processed/k729_2022_cuid_lat_lon.csv')
df_k729_2022_cuid_grouped = pd.read_csv('../../data/processed/k729_2022_cuid_grouped_lat_lon.csv')
# 2. transform values to numbers
df_k729_2022_cuid_grouped['x'] = df_k729_2022_cuid_grouped['x'].apply(lambda x: ast.literal_eval(x))
df_k729_2022_cuid_grouped['y'] = df_k729_2022_cuid_grouped['y'].apply(lambda y: ast.literal_eval(y))
df_k729_2022_cuid_grouped['vx'] = df_k729_2022_cuid_grouped['vx'].apply(lambda vx: ast.literal_eval(vx))
df_k729_2022_cuid_grouped['vy'] = df_k729_2022_cuid_grouped['vy'].apply(lambda vy: ast.literal_eval(vy))
df_k729_2022_cuid_grouped['lat'] = df_k729_2022_cuid_grouped['lat'].apply(lambda lat: ast.literal_eval(lat))
df_k729_2022_cuid_grouped['lon'] = df_k729_2022_cuid_grouped['lon'].apply(lambda lon: ast.literal_eval(lon))
# 3. create tuple lists
list_k729_2022_x_y_tuples = get_x_y_tuple_list(df_k729_2022_cuid_grouped, ['x','y'])
k729_2022_vx_vy_tuples = get_x_y_tuple_list(df_k729_2022_cuid_grouped, ['vx','vy'])
k729_2022_lat_lon_tuples = get_x_y_tuple_list(df_k729_2022_cuid_grouped,['lat','lon'])

In [ ]:
# create a list of all the first x-y coordinates of each row and a list for the last entries
first_x_y = [x_y[0] for x_y in list_k729_2022_x_y_tuples]
last_x_y = [x_y[-1] for x_y in list_k729_2022_x_y_tuples]

# make a list of all x coordinates and a list of all y coordinates using both lists
x_values_first = [x for x,y in first_x_y]
y_values_first = [y for x,y in first_x_y]
x_values_last = [x for x,y in last_x_y]
y_values_last = [y for x,y in last_x_y]

# add both x lists together
x_values_first.extend(x_values_last)
y_values_first.extend(y_values_last)

bins = 6**2
heatmap, xedges, yedges = np.histogram2d(x_values_first, y_values_first, bins=bins)

# Plot the heatmap
plt.imshow(heatmap.T, origin='lower', cmap='hot', interpolation='nearest')
plt.colorbar(label='Density')
plt.title('Heatmap of Start and Endpoints of Vehicle Tracks')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.show()

In [ ]:
import seaborn as sns
from src.features.get_global_dataframe_limits import get_global_dataframe_limits
sns.kdeplot(x=x_values_first, y=y_values_first, cmap="Blues", fill=True, thresh=0.05)
limits = get_global_dataframe_limits(df_k729_2022_cuid, ['x','y'])
plt.xlim(limits['x_min'], limits['x_max'])
plt.ylim(limits['y_min'], limits['y_max'])
plt.title('KDE Plot of X-Y Values')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.show()

## density for other intersections

In [ ]:
from sklearn.cluster import OPTICS
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt

## data import

In [ ]:
from src.features.get_x_y_tuples import get_x_y_tuple_list
from src.features.get_first_and_last_x_y_coordinates import get_first_and_last_x_y_coordinates 
from src.features.get_first_x_y_coordinates import get_first_x_y_coordinates
from src.features.get_last_x_y_coordinates import get_last_x_y_coordinates
def import_data(df_path):
    df_cuid = pd.read_csv(df_path)
    df_cuid_grouped_path = df_path.replace('.csv', '_grouped.csv')
    df_cuid_grouped = pd.read_csv(df_cuid_grouped_path)
    # 1.1 CONVERT FEATURES TO NUMBERS
    df_cuid_grouped['x'] = df_cuid_grouped['x'].apply(lambda x: ast.literal_eval(x))
    df_cuid_grouped['y'] = df_cuid_grouped['y'].apply(lambda y: ast.literal_eval(y))
    list_x_y_tuples = get_x_y_tuple_list(df_cuid_grouped, ['x','y'])
    first_last_x_coords, first_last_y_coords = get_first_and_last_x_y_coordinates(list_x_y_tuples)
    X = np.array([first_last_x_coords, first_last_y_coords]).T

    first_x_coords, first_y_coords = get_first_x_y_coordinates(list_x_y_tuples)
    first_coordinates = np.array([first_x_coords, first_y_coords]).T

    last_x_coords, last_y_coords = get_last_x_y_coordinates(list_x_y_tuples)
    last_coordinates = np.array([last_x_coords, last_y_coords]).T
    return df_cuid, df_cuid_grouped, X, first_coordinates, last_coordinates, list_x_y_tuples
        

In [ ]:
df_path_k729_2022 = f'{parent_parent_dir}/data/processed/k729_2022_cuid.csv'
df_path_k733_2020 = f'{parent_parent_dir}/data/processed/k733_2020_cuid.csv'
df_path_k733_2018 = f'{parent_parent_dir}/data/processed/k733_2018_cuid.csv'

df_cuid_k729_2022, df_cuid_grouped_k729_2022, X_k729_2022, first_coordinates_k729_2022, last_coordinates_k729_2022, list_x_y_tuples_k729_2022 = import_data(df_path_k729_2022)
df_cuid_k733_2020, df_cuid_grouped_k733_2020, X_k733_2020, first_coordinates_k733_2020, last_coordinates_k733_2020, list_x_y_tuples_k733_2020 = import_data(df_path_k733_2020)
df_cuid_k733_2018, df_cuid_grouped_k733_2018, X_k733_2018, first_coordinates_k733_2018, last_coordinates_k733_2018, list_x_y_tuples_k733_2018 = import_data(df_path_k733_2018)

In [ ]:
def plot_density_heatmap(list_xy_tuples):
    # create a list of all the first x-y coordinates of each row and a list for the last entries
    first_x_y = [x_y[0] for x_y in list_xy_tuples]
    last_x_y = [x_y[-1] for x_y in list_xy_tuples]

    # make a list of all x coordinates and a list of all y coordinates using both lists
    x_values_first = [x for x,y in first_x_y]
    y_values_first = [y for x,y in first_x_y]
    x_values_last = [x for x,y in last_x_y]
    y_values_last = [y for x,y in last_x_y]

    # add both x lists together
    x_values_first.extend(x_values_last)
    y_values_first.extend(y_values_last)

    bins = 6**2
    heatmap, xedges, yedges = np.histogram2d(x_values_first, y_values_first, bins=bins)

    # Plot the heatmap
    plt.imshow(heatmap.T, origin='lower', cmap='hot', interpolation='nearest')
    plt.colorbar(label='Density')
    plt.title('Heatmap of Start and Endpoints of Vehicle Tracks')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.show()

In [ ]:
plot_density_heatmap(list_x_y_tuples_k729_2022)

In [ ]:
plot_density_heatmap(list_x_y_tuples_k733_2020)

In [ ]:
plot_density_heatmap(list_x_y_tuples_k733_2018)